# Merge Dask DataFrames

To do:
- Add some markdown
- use default coiled software env

## Launch Coiled Cluster

In [1]:
import coiled

In [2]:
import pandas as pd
import dask.dataframe as dd
import dask

In [3]:
cluster = coiled.Cluster(
    name="dask-merge",
    n_workers=50,
    backend_options={'spot':'True'},
    software='coiled-examples/numpy-zarr',
    shutdown_on_close=False,
)

/Users/rpelgrim/mambaforge/envs/numpy-zarr/lib/python3.9/site-packages/rich/live.py:221: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Using existing cluster: 'dask-merge'

In [4]:
from distributed import Client
client = Client(cluster)
client

/Users/rpelgrim/mambaforge/envs/numpy-zarr/lib/python3.9/site-packages/distributed/client.py:1128: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| msgpack | 1.0.3  | 1.0.2     | 1.0.2   |
+---------+--------+-----------+---------+
Notes: 
-  msgpack: Variation is ok, as long as everything is above 0.6
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


<Client: 'tls://10.4.7.75:8786' processes=49 threads=98, memory=375.69 GiB>

## Create Datasets

In [8]:
# create large dataset as parquet to S3
def submit_jobs():
    from distributed import get_client
    
    with get_client() as client:        
        large = dask.datasets.timeseries(start="1990-01-01", end="2020-01-01", freq="1s", partition_freq="1M")
        
        large.to_parquet(
            's3://coiled-datasets/dask-merge/large.parquet',
            engine="fastparquet"
        )

client.submit(submit_jobs).result()

In [17]:
# write small dataset as parquet to S3
def submit_jobs():
    from distributed import get_client
    
    with get_client() as client:        
        small = dask.datasets.timeseries(start="1990-01-31", end="2020-01-01", freq="1D", dtypes={"z": int}).compute()
        
        small.to_parquet(
            's3://coiled-datasets/dask-merge/small.parquet'
        )

client.submit(submit_jobs).result()

## Merge Large Dask DataFrame to small pandas Dataframe

In [18]:
large = dd.read_parquet('s3://coiled-datasets/dask-merge/large.parquet')
small = pd.read_parquet('s3://coiled-datasets/dask-merge/small.parquet')

In [10]:
large.npartitions

359

In [11]:
large.head()

,id,name,x,y
timestamp,,,,
1990-01-31 00:00:00,964,Wendy,-0.588935,0.587432
1990-01-31 00:00:01,981,George,0.657202,-0.871205
1990-01-31 00:00:02,948,Edith,-0.904222,0.245642
1990-01-31 00:00:03,1052,Tim,-0.612337,0.072341
1990-01-31 00:00:04,998,George,-0.638432,-0.194386


In [22]:
small.head()

,z
timestamp,
1990-01-31,1001
1990-02-01,1038
1990-02-02,999
1990-02-03,1006
1990-02-04,1018


In [20]:
join = large.merge(
    small, 
    how="left", 
    on=["timestamp"]
)

In [21]:
join.head()

,id,name,x,y,z
timestamp,,,,,
1990-01-31 00:00:00,964,Wendy,-0.588935,0.587432,1001.0
1990-01-31 00:00:01,981,George,0.657202,-0.871205,NaN
1990-01-31 00:00:02,948,Edith,-0.904222,0.245642,NaN
1990-01-31 00:00:03,1052,Tim,-0.612337,0.072341,NaN
1990-01-31 00:00:04,998,George,-0.638432,-0.194386,NaN


In [15]:
join.loc["1990-02-01"].compute()

,id,name,x,y,z
timestamp,,,,,
1990-02-01 00:00:00,1029,Alice,0.951480,0.542765,1000.0
1990-02-01 00:00:01,1006,Laura,0.110207,-0.585922,NaN
1990-02-01 00:00:02,997,George,0.332245,-0.630354,NaN
1990-02-01 00:00:03,1023,Laura,0.702961,0.258879,NaN
1990-02-01 00:00:04,981,George,-0.134436,-0.890895,NaN
...,...,...,...,...,...
1990-02-01 23:59:55,993,Xavier,-0.732635,-0.348656,NaN
1990-02-01 23:59:56,1003,Hannah,0.801193,0.440596,NaN
1990-02-01 23:59:57,1064,Tim,0.580918,-0.652578,NaN


## Benchmark Dask+small_Dask vs Dask+pandas vs Dask+small_dask(>1 partition)

In [21]:
client.restart()

<Client: 'tls://10.4.7.75:8786' processes=0 threads=0, memory=0 B>

In [22]:
large = dd.read_parquet('s3://coiled-datasets/dask-merge/large.parquet')
small_df = pd.read_parquet('s3://coiled-datasets/dask-merge/small.parquet')
small_ddf_1p = dd.from_pandas(small_df, npartitions=1)
small_ddf_10p = dd.from_pandas(small_df, npartitions=10)

In [8]:
large

,id,name,x,y
npartitions=359,,,,
1990-01-31 00:00:00,int64,object,float64,float64
1990-02-28 00:00:00,...,...,...,...
...,...,...,...,...
2019-11-30 00:00:00,...,...,...,...
2019-12-30 23:59:59,...,...,...,...


In [9]:
small_df

,z
timestamp,
1990-01-31,1001
1990-02-01,1038
1990-02-02,999
1990-02-03,1006
1990-02-04,1018
...,...
2019-12-27,995
2019-12-28,964
2019-12-29,1051


In [10]:
small_ddf_1p

,z
npartitions=1,
1990-01-31,int64
2019-12-31,...


In [11]:
small_ddf_10p

,z
npartitions=10,
1990-01-31,int64
1993-01-28,...
...,...
2017-01-06,...
2019-12-31,...


In [13]:
import distributed

In [45]:
%%time
# merge dask dataframe with small pandas dataframe
join1 = large.merge(
    small_df, 
    how="left", 
    on=["timestamp"]
)

join1.head(10000)

CPU times: user 102 ms, sys: 10 ms, total: 112 ms
Wall time: 10.7 s


,id,name,x,y,z
timestamp,,,,,
1990-01-31 00:00:00,964,Wendy,-0.588935,0.587432,1001.0
1990-01-31 00:00:01,981,George,0.657202,-0.871205,NaN
1990-01-31 00:00:02,948,Edith,-0.904222,0.245642,NaN
1990-01-31 00:00:03,1052,Tim,-0.612337,0.072341,NaN
1990-01-31 00:00:04,998,George,-0.638432,-0.194386,NaN
...,...,...,...,...,...
1990-01-31 02:46:35,987,Sarah,-0.853707,-0.681082,NaN
1990-01-31 02:46:36,995,Xavier,-0.656976,0.117241,NaN
1990-01-31 02:46:37,1037,Sarah,0.320692,0.711906,NaN


In [ ]:
%%time
# merge dask dataframe with small pandas dataframe
join2 = large.merge(
    small_ddf_1p, 
    how="left", 
    on=["timestamp"]
)

join2.head(10000)

In [ ]:
%%time
# merge dask dataframe with small pandas dataframe
join2 = large.merge(
    small_ddf_1p, 
    how="left", 
    on=["timestamp"]
)

join2.head(10000)

In [17]:
%%time
# merge dask dataframe with small pandas dataframe
join1 = large.merge(
    small_df, 
    how="left", 
    on=["timestamp"]
).persist()

distributed.wait(join1)

CPU times: user 530 ms, sys: 45.3 ms, total: 575 ms
Wall time: 38.5 s


DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('join-indexed-500bf3b7a42bf7ee549d8e8ced867201', 150)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('join-indexed-500bf3b7a42bf7ee549d8e8ced867201', 271)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('join-indexed-500bf3b7a42bf7ee549d8e8ced867201', 40)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('join-indexed-500bf3b7a42bf7ee549d8e8ced867201', 100)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('join-indexed-500bf3b7a42bf7ee549d8e8ced867201', 109)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('join-indexed-500bf3b7a42bf7ee549d8e8ced867201', 159)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('join-indexed-500bf3b7a42bf7ee549d8e8ced867201', 49)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('join-indexed-500bf3b7a42bf7ee549d8e8ced867201', 228)>, <Future: finished, type: pandas.core.frame.Dat

In [20]:
%%time
# merge dask dataframe with small dask dataframe (1 partition)
join2 = large.merge(
    small_ddf_1p, 
    how="left", 
    on=["timestamp"]
).persist()

distributed.wait(join2)

CPU times: user 475 ms, sys: 43 ms, total: 518 ms
Wall time: 32.7 s


DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('join-indexed-500bf3b7a42bf7ee549d8e8ced867201', 112)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('join-indexed-500bf3b7a42bf7ee549d8e8ced867201', 254)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('join-indexed-500bf3b7a42bf7ee549d8e8ced867201', 2)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('join-indexed-500bf3b7a42bf7ee549d8e8ced867201', 25)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('join-indexed-500bf3b7a42bf7ee549d8e8ced867201', 121)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('join-indexed-500bf3b7a42bf7ee549d8e8ced867201', 144)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('join-indexed-500bf3b7a42bf7ee549d8e8ced867201', 240)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('join-indexed-500bf3b7a42bf7ee549d8e8ced867201', 263)>, <Future: finished, type: pandas.core.frame.Data

In [23]:
%%time
# merge dask dataframe with small dask dataframe (10 partitions)
join3 = large.merge(
    small_ddf_10p, 
    how="left", 
    on=["timestamp"]
).persist()

distributed.wait(join3)

CPU times: user 483 ms, sys: 46 ms, total: 529 ms
Wall time: 32.6 s


DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('join-indexed-0dd3b843d71c3e61d8e20e73f782915e', 368)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('join-indexed-0dd3b843d71c3e61d8e20e73f782915e', 258)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('join-indexed-0dd3b843d71c3e61d8e20e73f782915e', 267)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('join-indexed-0dd3b843d71c3e61d8e20e73f782915e', 96)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('join-indexed-0dd3b843d71c3e61d8e20e73f782915e', 105)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('join-indexed-0dd3b843d71c3e61d8e20e73f782915e', 224)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('join-indexed-0dd3b843d71c3e61d8e20e73f782915e', 114)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('join-indexed-0dd3b843d71c3e61d8e20e73f782915e', 233)>, <Future: finished, type: pandas.core.frame.Da

distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
distributed.deploy.cluster - WARNING - Failed to sync cluster info multiple times - perhaps there's a connection issue? Error:
Traceback (most recent call last):
  File "/Users/rpelgrim/mambaforge/envs/numpy-zarr/lib/python3.9/site-packages/distributed/comm/tcp.py", line 398, in connect
    stream = await self.client.connect(
  File "/Users/rpelgrim/mambaforge/envs/numpy-zarr/lib/python3.9/site-packages/tornado/tcpclient.py", line 275, in connect
    af, addr, stream = await connector.start(connect_timeout=timeout)
asyncio.exceptions.CancelledError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/rpelgrim/mambaforge/envs/numpy-zarr/lib/python3.9/asyncio/tasks.py", line 492, in w

## Merge two Dask DataFrames

In [8]:
left = dask.datasets.timeseries(dtypes={"foo": int})
right_one = dask.datasets.timeseries(dtypes={"bar": int})
right_two = dask.datasets.timeseries(dtypes={"baz": int})

In [9]:
# timeseries returns a dataframe indexed by
# timestamp, we don't need to set_index.

# left.set_index("timestamp")

#left.to_parquet("left", overwrite=True)
#left = dd.read_parquet("left")

# If the dataframe can fit in RAM, you can also use persist

# left = left.persist()


result1 = left.merge(
    right_one, how="left", left_index=True, right_index=True)

In [13]:
result2 = result1.merge(
    right_two, how="left", left_index=True, right_index=True)

In [11]:
left.head()

,foo
timestamp,
2000-01-01 00:00:00,1032
2000-01-01 00:00:01,993
2000-01-01 00:00:02,993
2000-01-01 00:00:03,995
2000-01-01 00:00:04,995


In [10]:
result1.head()

,foo,bar
timestamp,,
2000-01-01 00:00:00,1032,1016
2000-01-01 00:00:01,993,949
2000-01-01 00:00:02,993,1004
2000-01-01 00:00:03,995,1023
2000-01-01 00:00:04,995,1042


In [14]:
result2.head()

,foo,bar,baz
timestamp,,,
2000-01-01 00:00:00,1032,1016,1014
2000-01-01 00:00:01,993,949,1010
2000-01-01 00:00:02,993,1004,984
2000-01-01 00:00:03,995,1023,1005
2000-01-01 00:00:04,995,1042,981


## Simple Example

In [35]:
import dask.dataframe as dd
import pandas as pd
df_large = pd.DataFrame(
    {
        "Name": ["Azza", "Brandon", "Cedric", "Devonte", "Eli", "Fabio"], 
        "Age": [29, 30, 21, 57, 32, 19]
    }
)
large = dd.from_pandas(df_large, npartitions=2)

In [36]:
small = pd.DataFrame(
    {
        "Name": ["Azza", "Cedric", "Fabio"], 
        "City": ["Beirut", "Dublin", "Rosario"]
    }
)

In [40]:
join = large.merge(small, how="left", on=["Name"])

In [41]:
join.compute()

,Name,Age,City
0,Azza,29,Beirut
1,Brandon,30,NaN
2,Cedric,21,Dublin
0,Devonte,57,NaN
1,Eli,32,NaN
2,Fabio,19,Rosario
